Imports
-> MNE is the main library for EEG signal processing in Python. It has almost all the functionality from EEGLab-Matlab and you can import EEGLab set files with a function.


In [1]:
import os
import numpy as np
import mne

Load data: <br>
Load the calibration.set file using MNE. 

In [2]:
calibration_path = "../BSPM/data/calibration.set"
eeg = mne.io.read_raw_eeglab(calibration_path,preload=True)
data, times = eeg[:, :]

Reading ../BSPM/data/calibration.fdt
Reading 0 ... 248153  =      0.000 ...   969.348 secs...


Downsample the data: <br>
In the next step we downsample the data by 8 factors in order to process the data more efficiently.


In [3]:
eeg.resample(eeg.info['sfreq']/8) # Downsample the data by 8.

<RawEEGLAB  |  calibration.fdt, n_channels x n_times : 27 x 31019 (969.3 sec), ~6.5 MB, data loaded>

Extract epoches of the events. <br>
What we want is to get the epochs starting from 0.2 secs after the event marker and ending at the 0.8 secs. <br>  However due to the MNE implementation it is not possible, which is why we start at 0.2 secs before the event.
We epoch only the events labeled S4: No Error and S5: Error and extract the data for further processing.


In [4]:
(events_from_annot,event_dict) = mne.events_from_annotations(eeg)
epochs_noError = mne.Epochs(eeg,events_from_annot,tmin=-0.2,tmax=0.8,event_id=7,preload=True) # S 4
epochs_Error = mne.Epochs(eeg,events_from_annot,tmin=-0.2,tmax=0.8,event_id=8,preload=True) # S 5
noError_data = epochs_noError.get_data()
error_data = epochs_Error.get_data()

Used Annotations descriptions: ['R  1', 'R  2', 'R  3', 'S  1', 'S  2', 'S  3', 'S  4', 'S  5', 'S  7', 'S  8', 'S  9', 'empty']
190 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 190 events and 33 original time points ...
0 bad epochs dropped
110 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 110 events and 33 original time points ...
0 bad epochs dropped


We actually want the two points 210.9 and 285.2, unfortunately tmin cannot be larger or equal than 0.0 therefore we have to start epoching before the event. <br>
<br>
The epoching process tells us it gathered 33 events between -0.2 and 0.8. That is a time distance of 1. We need the time points at 0.2109 and 0.2852. <br>
<br>
We do a simple ratio calculation for both: <br>
Timescale of 1.0 seconds is divided into 33 discreet time points <br>
1.0/33 = 0.03 seconds of precision. <br>
For the timepoint of 0.2109: <br> 
-0.2 + 0.03x = 0.2109 <br>
0.03x = 0.4109 <br>
x = 0.4109 / 0.03 = 13.69 ~ index 14 <br>
For the timepoint of 0.2852: <br>
-0.2 + 0.03x = 0.2852 <br>
0.03x = 0.4852 <br>
x = 0.4852/0.03 = 16.17 ~ index 16 <br>


In [5]:
noError_data_210 = noError_data[:,:,14]
noError_data_285 = noError_data[:,:,16]
noError_data = np.concatenate((noError_data_210,noError_data_285), axis=1)
error_data_210 = error_data[:,:,14]
error_data_285 = error_data[:,:,16]
error_data = np.concatenate((error_data_210,error_data_285), axis=1)

Generate data labels: <br>
For no error use label 1 and for error events use label -1.


In [6]:
noError_label = np.ones(190)
error_label = np.ones(110)*-1

Add the data and labels together: <br>
Combine the data from no error and error epoches into a single 2D array. <br>
Combine the labels into a single 1D array where the indexes match that of the data.


In [7]:
all_data = np.concatenate((noError_data,error_data),axis=0)
all_labels = np.concatenate((noError_label,error_label),axis=0)

Evaluate discriminative power of the features using fisher rank.

In [8]:
import seaborn as sns
sns.set()
from skfeature.function.similarity_based import fisher_score
score = fisher_score.fisher_score(all_data, all_labels)
rank = fisher_score.feature_ranking(score)
score = np.flip(np.sort(score))
x_axis = np.arange(score.shape[0])
import matplotlib.pyplot as plt
%matplotlib qt
plt.figure()
plt.plot(x_axis,score)
plt.xlabel("Components")
plt.ylabel("Fisher's score")
plt.title("Fisher's scores for components")
plt.show()

Print first and second features using grouping. <br>
As our features are super small, we have to normalize the data using minmax scaling. <br>
In order to do a group scatter plot (gscatter in matlab), use a pandas data frame.

In [9]:
first_feature = all_data[:,rank[0]]
first_feature.reshape(1,-1)
first_feature = (first_feature - first_feature.min()) / (first_feature.max() - first_feature.min())
second_feature = all_data[:,rank[1]]
second_feature.reshape(1,-1)
second_feature = (second_feature - second_feature.min()) / (second_feature.max() - second_feature.min())

import pandas as pd
data = pd.DataFrame({"First feature": first_feature, "Second feature": second_feature, "Category": all_labels})
groups = data.groupby("Category")
plt.figure()
plt.xlabel("1st feature")
plt.ylabel("2nd feature")
for name, group in groups:
    plt.plot(group["First feature"], group["Second feature"], marker="o", linestyle="", label=name, markersize=4)
plt.legend()

Rescale the data columns into the normal space

In [10]:
for i in range(all_data.shape[1]):
    all_data[:,i] = (all_data[:,i] - all_data[:,i].min()) / (all_data[:,i].max() - all_data[:,i].min())

Here split the data into pieces with K-fold K-times crossvalidation for K=10

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(X=all_data,y=all_labels)

10

Train a classifier: (Shrink LDA with Shrinkage = 0.4)

In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
general_score = 0.0
for train_index, test_index in kf.split(all_data):
    clf = LinearDiscriminantAnalysis(solver='lsqr',shrinkage=0.4)
    clf.fit(all_data[train_index], all_labels[train_index])
    score = clf.score(all_data[test_index], all_labels[test_index])
    general_score += score
    print("Score is: " + str(score))
print("Mean score is: " +str(general_score/10.0))

Score is: 0.9
Score is: 0.9333333333333333
Score is: 0.8333333333333334
Score is: 0.8
Score is: 0.8666666666666667
Score is: 0.8
Score is: 0.7333333333333333
Score is: 0.9
Score is: 0.8
Score is: 0.6666666666666666
Mean score is: 0.8233333333333335


Get labels on recall.set

In [13]:
recall_path = "../BSPM/data/recall.set"
eeg_recall = mne.io.read_raw_eeglab(recall_path,preload=True)
data, times = eeg_recall[:, :]
eeg_recall.resample(eeg_recall.info['sfreq']/8) # Downsample the data by 8.
(events_from_annot,event_dict) = mne.events_from_annotations(eeg_recall)
print(event_dict)
#We require S 6: 7
#7 is the event id with which we need to epoch from
epochs_s6 = mne.Epochs(eeg_recall,events_from_annot,tmin=-0.2,tmax=0.8,event_id=7,preload=True, reject_by_annotation=False) # S 6
epochs_data = epochs_s6.get_data()
epochs_s6_210 = epochs_data[:,:,14]
epochs_s6_285 = epochs_data[:,:,16]
epochs_all_data = np.concatenate((epochs_s6_210,epochs_s6_285), axis=1)
#Min Max Scaling on the epochs on recall.set
for i in range(epochs_all_data.shape[1]):
    epochs_all_data[:,i] = (epochs_all_data[:,i] - epochs_all_data[:,i].min()) / (epochs_all_data[:,i].max() - epochs_all_data[:,i].min())
#Train a ShrinkLDA classifier on calibration.set
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis(solver='lsqr',shrinkage=0.4)
clf.fit(all_data, all_labels)
predicted = clf.predict(epochs_all_data)
import scipy.io as sio
sio.savemat('./lda.mat', {'vector_output': predicted})

Reading ../BSPM/data/recall.fdt
Reading 0 ... 268019  =      0.000 ...  1046.949 secs...
Used Annotations descriptions: ['R  1', 'R  2', 'R  3', 'S  1', 'S  2', 'S  3', 'S  6', 'S  7', 'S  8', 'S  9', 'empty']
{'R  1': 1, 'R  2': 2, 'R  3': 3, 'S  1': 4, 'S  2': 5, 'S  3': 6, 'S  6': 7, 'S  7': 8, 'S  8': 9, 'S  9': 10, 'empty': 11}
300 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 300 events and 33 original time points ...
0 bad epochs dropped
